In [ ]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc
import itertools
import importlib

# BOKEH
import bokeh.plotting as bk
import bokeh.models as bkmod
import bokeh.layouts as bklay
import bokeh.palettes as bkpalettes
import bokeh.util.hex as bkhex
import bokeh.transform as bktrfm
import bokeh.colors as bkcolors

bk.output_notebook()

# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp

# Custom imports
import bokeh_tools as bktools
import Presets as bkpresets

# BBStudies
import sys
sys.path.append('/Users/pbelanger/ABPLocal/BBStudies')
import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst



# Setting default values
#------------------------------------------------
BOKEH_FIGS = {}
# _default_fig_width  = 2000
_default_fig_width  = 1500
_default_fig_height = 400
_default_fig_pad    = 100


# tracking_path = 'zfruits/BBB_Signature/FULL/'
# tracked   = xPlus.Tracking_Interface.from_parquet(tracking_path ,partition_name='CHUNK')

data_path = '../003_particle_dist_and_track/zfruits/BBB_Signature_V2/DATA/'
data      = xPlus.Tracking_Interface.from_parquet(data_path,partition_name='BUNCH',partition_ID='0220')

_cpt      = xPlus.Tracking_Interface.from_parquet(data_path.replace('DATA','CHECKPOINTS'),partition_name='BUNCH',partition_ID='0220')
data._checkpoint = _cpt._checkpoint

# _trk      = xPlus.Tracking_Interface.from_parquet(data_path.replace('DATA','FULL/BUNCH_0220'),partition_name='CHUNK',handpick_particles = [1, 9, 536, 4883])
# data._df          = _trk.df



#bk.show(bklay.column(BOKEH_FIGS['lattice'],BOKEH_FIGS['twiss']))

In [ ]:
def dict_to_html(dictionaries, headers, margin=10, width=int(_default_fig_width)-10,force_width=120):
    def format_value(key,value):
        if isinstance(value, dict):
            return dict_to_html([value], headers=[''], margin=0, width=width,force_width=force_width)
        return str(value)

    html_content = ""
    for i, d in enumerate(dictionaries):
        # Add a tab character before the div content
        html_content += f"\t<div style='background-color: black; color: white; padding: 10px; border-radius: 5px; margin: 0 0 0 {margin}px; width: {width}px;'><h2>{headers[i]}</h2><ul style='list-style-type: none; padding: 0; margin: 0;'>"
        for key, value in d.items():
            # Use flexbox for better alignment
            if force_width:
                html_content += f"\t\t<li style='line-height: 1; display: flex;'><div style='width: {force_width}px;'><strong>{key}:</strong></div><div style='padding-left: 10px;'>{format_value(key,value)}</div></li>"
            else:
                html_content += f"\t\t<li style='line-height: 1; display: flex;'><div><strong>{key}:</strong></div><div style='padding-left: 10px;'>{format_value(key,value)}</div></li>"
        html_content += "</ul></div>"
        if i < len(dictionaries) - 1:
            # Add a tab character before the horizontal line
            html_content += "\t<hr style='margin: 5px;'>"
    return html_content


import ruamel.yaml
ryaml = ruamel.yaml.YAML()
def read_configuration(config_path="config.yaml"):
    # Read configuration for simulations
    with open(config_path, "r") as fid:
        config = ryaml.load(fid)


    return config


config = read_configuration('../003_particle_dist_and_track/config.yaml')
config_collider = read_configuration('../001_configure_collider/config.yaml')['config_collider']

config_info = bkmod.Div(text=dict_to_html([config],['Collider config.yaml']),width=_default_fig_width, height=1000)
config_collider_info = bkmod.Div(text=dict_to_html([config_collider],['Collider config.yaml'],force_width=0),width=_default_fig_width, height=1000)


metadata = data.to_dict()
metadata.pop('parquet_data');
metadata.pop('W_matrix');
metadata.pop('particle_on_co');
tracking_info = bkmod.Div(text=dict_to_html([metadata],['Tracking Info']),width=_default_fig_width, height=300)



bk.show(bklay.column(config_info))
# bk.show(bklay.column(config_collider_info))
# bk.show(bklay.column(tracking_info))

In [ ]:
(data.data.stop_at_turn - data.data.start_at_turn).max()

In [ ]:
config['tracking']

In [ ]:
data.to_dict().pop('W_matrix')

metadata = data.to_dict()
metadata.pop('parquet_data');
metadata.pop('W_matrix');
metadata.pop('particle_on_co');
metadata

In [ ]:
def return_name(group):
    return group.name

data.data.groupby('Chunk ID').apply(return_name)

In [ ]:
# Extraction action
J_df = data.checkpoint[['BUNCH','Chunk ID','turn','particle']]
J_df.insert(4,'Jx/emitt',1/2 * (data.checkpoint_sig.x_sig**2 + data.checkpoint_sig.px_sig**2))
J_df.insert(5,'Jy/emitt',1/2 * (data.checkpoint_sig.y_sig**2 + data.checkpoint_sig.py_sig**2))
J_df.dropna(inplace=True)

# Making Hextile grid
J_min  = 0
J_max  = 50
n_bins = 300

_size        = (J_max-J_min)/n_bins
_orientation = 'pointytop'


# Creating hextile template
x_corners = [J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max()]
y_corners = [J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max()]
XX,YY    = np.meshgrid(np.arange(J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max(),_size),
                    np.arange(J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max(),_size))
hextiles_template = bkhex.hexbin(np.array(list(XX.flatten())+x_corners), np.array(list(YY.flatten())+y_corners), size=_size,orientation=_orientation)
hextiles_template['counts']  = 0
hextiles_template = hextiles_template.rename(columns={'counts':'counts:active'}).set_index(['q','r'])

# Looping over chunks
for name,group in J_df.groupby('Chunk ID'):
    # Forcing corner values to have same grid.
    within_limits = (group['Jx/emitt'] < J_max)&(group['Jy/emitt'] < J_max)
    data_x = np.array(list(group['Jx/emitt'][within_limits]) + [J_min,J_max])
    data_y = np.array(list(group['Jy/emitt'][within_limits]) + [J_min,J_max])
    _hex = bkhex.hexbin(data_x,data_y, size=_size,orientation=_orientation)

    # Removing corner values
    _hex = _hex[1:-1]

    # Adding chunk ID
    hextiles_template.insert(name+1,f'counts:{name}',_hex.set_index(['q','r'])['counts'])


# setting empty bins to 0
# hextiles_template = hextiles_template.fillna(0).reset_index()
# hextiles_template = hextiles_template.reset_index()
hextiles_template.reset_index(inplace=True)
hextiles_template['counts:active'] = hextiles_template['counts:0']
hextiles_template



In [ ]:
data.compute_intensity(coll_opening=5)

In [ ]:
data.coord_sig

In [ ]:
# Reload bkpresets

importlib.reload(bkpresets)
BOKEH_FIGS['x-px'] = bkpresets.make_scatter_fig(data.coord_sig,xy=('x_sig','px_sig'),title='x norm. phase space',width=_default_fig_width//4,height=_default_fig_height)
BOKEH_FIGS['y-py'] = bkpresets.make_scatter_fig(data.coord_sig,xy=('y_sig','py_sig'),title='y norm. phase space',width=_default_fig_width//4,height=_default_fig_height)
BOKEH_FIGS['zeta-pzeta'] = bkpresets.make_scatter_fig(data.coord_sig,xy=('zeta_sig','pzeta_sig'),title='zeta norm. phase space',width=_default_fig_width//4,height=_default_fig_height)

BOKEH_FIGS['x-y'] = bkpresets.make_scatter_fig(data.data,xy=('x_min','y_min'),title='Transverse norm space',width=3*_default_fig_width//4,height=2*_default_fig_height)


bktools.set_aspect(BOKEH_FIGS['x-px']       , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['y-py']       , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['zeta-pzeta'] , x_lim=(-1,1),y_lim=(-1,1), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['x-y']        , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)



grid = bk.column([bklay.gridplot([[BOKEH_FIGS['x-px'] ,BOKEH_FIGS['y-py'] ,BOKEH_FIGS['zeta-pzeta']]],toolbar_location='right'),
                  bklay.gridplot([[BOKEH_FIGS['x-y'] ]],toolbar_location='right')])

# grid = bklay.layout([ [BOKEH_FIGS['x-px'] ,BOKEH_FIGS['y-py'] ,BOKEH_FIGS['zeta-pzeta']], 
#                         [BOKEH_FIGS['x-y'] ]],)
bk.show(grid)


In [ ]:
importlib.reload(bkpresets)
BOKEH_FIGS['Intensity'] = bkpresets.make_intensity_fig(data,title='Intensity',width=3*_default_fig_width//4,height=_default_fig_height)


grid = bklay.gridplot([[BOKEH_FIGS['Intensity']]],toolbar_location='right')
bk.show(grid)



In [ ]:
data.checkpoint

In [ ]:

def update_coll(_df,coll_opening = 5):
    
    # Resetting values
    #-----------------------------
    _width  = _df['width'].unique()[0]
    _height = _df['height'].unique()[0]
    _df['xs']   = 6*[np.array([0,0,_width,_width])]
    _df['ys']   = 6*[np.array([-_height/2,_height/2,_height/2,-_height/2])]

    # Distributing colls on x-axis
    #-----------------------------
    _df['xs'] += coll_opening*_df['sigma']
    _df.loc[_df.name.str.contains('left|bottom'),'xs']   *= -1

    # Rotating according to angle
    _x,_y  = np.stack(_df['xs']),np.stack(_df['ys'])
    _angle = np.stack(_df['angle'].apply(lambda angle: list(np.repeat(angle,4))))
    _x_rot = _x*np.cos(_angle) - _y*np.sin(_angle)
    _y_rot = _x*np.sin(_angle) + _y*np.cos(_angle)


    _df['xs'] = list(_x_rot)
    _df['ys'] = list(_y_rot)

    # Returning in mm
    # _df['xs'] *= 
    # _df['ys'] *= 

    return _df


def excursion_polygon(row):
    # skew col : y = ax + b
    skew_angle = 127.5
    a = np.tan(np.deg2rad(skew_angle-90))
    b = np.max([np.abs(row['skew_max']),np.abs(row['skew_min'])])/np.cos(np.deg2rad(skew_angle-90))

    # skew col : y = ax - b, x = (y+b)/a
    x1 = [row['x_max'],a*row['x_max'] - b]
    x2 = [(row['y_min']+b)/a,row['y_min']]

    # skew col : y = -ax - b, x = -(y+b)/a
    x3 = [-(row['y_min']+b)/a,row['y_min']]
    x4 = [row['x_min'],-a*row['x_min'] - b]

    # skew col : y = ax + b, x = (y-b)/a
    x5 = [row['x_min'],a*row['x_min'] + b]
    x6 = [(row['y_max']-b)/a,row['y_max']]

    # skew col : y = -ax + b, x = -(y-b)/a
    x7 = [-(row['y_max']-b)/a,row['y_max']]
    x8 = [row['x_max'],-a*row['x_max'] + b]

    if x5[1]<x4[1]:
        x4[1],x5[1] = x5[1],x4[1]
    
    if x8[1]<x1[1]:
        x1[1],x8[1] = x8[1],x1[1]

    return [x1[0],x2[0],x3[0],x4[0],x5[0],x6[0],x7[0],x8[0]], [x1[1],x2[1],x3[1],x4[1],x5[1],x6[1],x7[1],x8[1]]

#=========================================================================================================================
def make_collimation_fig(data,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,wheel_zoom",
                    active_drag     = "box_zoom",
                    active_scroll   = "wheel_zoom",
                    toolbar_location= "right")


    # Saving tools to tags
    # _palette = bkpalettes.Viridis8
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    # fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','@s'),(f'Value', '$y'),('Element','@name')])

    # Putting legend outside
    # fig.add_layout(bkmod.Legend(), 'right')
    #=====================================

    coll_opening = 5
    coll_alpha = np.deg2rad(127.5)
    pipe_r     = cst.LHC_W_BEAM_SCREEN/2
    coll_df    = pd.DataFrame({'name'   :  ['H_left','H_right','V_top','V_bottom','S_top','S_bottom'],
                               'width'  :  pipe_r*np.ones(6),
                               'height' :2*pipe_r*np.ones(6),
                               'xs'     :6*[np.zeros(4)],
                               'ys'     :6*[np.zeros(4)],
                                'angle' :[0,0,np.pi/2,np.pi/2,coll_alpha,coll_alpha],
                                'sigma' :[data.sig_x_coll,data.sig_x_coll,data.sig_y_coll,data.sig_y_coll,data.sig_skew_coll,data.sig_skew_coll]})

        
    coll_df = update_coll(coll_df,coll_opening=coll_opening)
    source  = bkmod.ColumnDataSource(coll_df)
    
    fig.patches(xs='xs', ys='ys',alpha=1,color='gray',source=source)


    # Crop at beam pipe
    _x_pipe = np.array([-2*pipe_r] + list(np.linspace(-pipe_r,pipe_r,200)) + [2*pipe_r])
    _y_pipe = np.sqrt(pipe_r**2 - _x_pipe**2)
    _y_pipe[np.abs(_x_pipe)>pipe_r] = 0
    fig.varea(x=_x_pipe,y1=2*pipe_r*np.ones(len(_x_pipe)),y2=_y_pipe  ,color='white',alpha=1)
    fig.varea(x=_x_pipe,y2=-2*pipe_r*np.ones(len(_x_pipe)),y1=-_y_pipe,color='white',alpha=1)


    # Adding particle trajectory:
    last_turn    = data.data.start_at_turn.max()
    last_turn_df = data.data.groupby('start_at_turn').get_group(last_turn).set_index('particle')

    sigma_bins     = [1,3,5,7,9,12]
    bin_values     = [np.sqrt(2*(i*data.sig_x)**2 + 2*(i*data.sig_y**2)) for i in sigma_bins]
    part_selection = pd.cut(np.sqrt(last_turn_df.x_max**2 +last_turn_df.px_max**2 + last_turn_df.y_max**2 +last_turn_df.py_max**2),bins=bin_values)
    part_chosen = []
    for name, group in part_selection.groupby(part_selection,observed=True):
        part_chosen.append(group.index.values[1])

    # part_chosen = part_chosen[::2]
    print(part_chosen)
    # part_chosen =[part_chosen[2]]
    for part_idx,color in zip(part_chosen,fig.tags[1]['palette']):
        _part = data.data.groupby('particle').get_group(part_idx)
        polygon = pd.DataFrame(_part.apply(lambda row:excursion_polygon(row),axis=1).to_list(),columns=['x','y'])


        source = bkmod.ColumnDataSource(pd.DataFrame({'x':polygon.x[-1:].sum(),'y':polygon.y[-1:].sum()}))
        fig.line('x','y', alpha=0.8,color=color,legend_label= f'Particle {part_idx}',source=source)
        # fig.scatter('x','y', alpha=0.8,color=color, source=source)

    # for part_idx,color in zip(part_chosen,fig.tags[1]['palette']):
    #     _part = data.checkpoint.groupby('particle').get_group(part_idx)

    #     source = bkmod.ColumnDataSource(_part[['x','y']])
    #     fig.line('x','y', alpha=0.6,color=color, source=source)
    #     fig.scatter('x','y', alpha=0.6,color=color, source=source)


    # for part_idx,color in zip(part_chosen,fig.tags[1]['palette']):
    #     _part = data.checkpoint.groupby('particle').get_group(part_idx)

    #     source = bkmod.ColumnDataSource(_part[['x','y']])
    #     fig.scatter('x','y', alpha=0.6,color=color, source=source)

    # fig.rect(x=[0], y=[0], width=0.00005, height=40, color="black",angle=np.deg2rad(127.5-90))
    # fig.rect(x=[0], y=[0], width=0.00005, height=40, color="black",angle=np.deg2rad(180-127.5-90))
    # fig.scatter('0','y_max', alpha=0.3,color='blue', source=source)
    # fig.scatter('x_max','0', alpha=0.3,color='blue', source=source)
    # fig.scatter('x_min','0', alpha=0.3,color='blue', source=source)
    # Axis and Legend
    #=====================================

    fig.xaxis.axis_label = 'x [m]'
    fig.yaxis.axis_label = 'y [m]'
    fig.legend.title     = r'Particles ID'
    fig.legend.click_policy="hide"
    # fig.x_range=bkmod.Range1d(-6, 6)
    # fig.y_range=bkmod.Range1d(-6, 6)
    # fig.match_aspect=True

    #=====================================

    return fig

fig = make_collimation_fig(data,title='Collimation',width=1000,height=700)
bktools.set_aspect(fig , x_lim=(-3e-3,3e-3),y_lim=(-3e-3,3e-3), aspect=1, margin=0)
bk.show(fig)

In [ ]:
data.data

In [ ]:

def update_coll(_df,coll_opening = 5):
    
    # Resetting values
    #-----------------------------
    _width  = _df['width'].unique()[0]
    _height = _df['height'].unique()[0]
    _df['xs']   = 6*[np.array([0,0,_width,_width])]
    _df['ys']   = 6*[np.array([-_height/2,_height/2,_height/2,-_height/2])]

    # Distributing colls on x-axis
    #-----------------------------
    _df['xs'] += coll_opening*_df['sigma']
    _df.loc[_df.name.str.contains('left|bottom'),'xs']   *= -1

    # Rotating according to angle
    _x,_y  = np.stack(_df['xs']),np.stack(_df['ys'])
    _angle = np.stack(_df['angle'].apply(lambda angle: list(np.repeat(angle,4))))
    _x_rot = _x*np.cos(_angle) - _y*np.sin(_angle)
    _y_rot = _x*np.sin(_angle) + _y*np.cos(_angle)


    _df['xs'] = list(_x_rot)
    _df['ys'] = list(_y_rot)

    # Returning in mm
    # _df['xs'] *= 
    # _df['ys'] *= 

    return _df


def excursion_polygon(row):
    # skew col : y = ax + b
    skew_angle = 127.5
    a = np.tan(np.deg2rad(skew_angle-90))
    b = np.max([np.abs(row['skew_max']),np.abs(row['skew_min'])])/np.cos(np.deg2rad(skew_angle-90))

    # skew col : y = ax - b, x = (y+b)/a
    x1 = [row['x_max'],a*row['x_max'] - b]
    x2 = [(row['y_min']+b)/a,row['y_min']]

    # skew col : y = -ax - b, x = -(y+b)/a
    x3 = [-(row['y_min']+b)/a,row['y_min']]
    x4 = [row['x_min'],-a*row['x_min'] - b]

    # skew col : y = ax + b, x = (y-b)/a
    x5 = [row['x_min'],a*row['x_min'] + b]
    x6 = [(row['y_max']-b)/a,row['y_max']]

    # skew col : y = -ax + b, x = -(y-b)/a
    x7 = [-(row['y_max']-b)/a,row['y_max']]
    x8 = [row['x_max'],-a*row['x_max'] + b]

    if x5[1]<x4[1]:
        x4[1],x5[1] = x5[1],x4[1]
    
    if x8[1]<x1[1]:
        x1[1],x8[1] = x8[1],x1[1]

    return [x1[0],x2[0],x3[0],x4[0],x5[0],x6[0],x7[0],x8[0]], [x1[1],x2[1],x3[1],x4[1],x5[1],x6[1],x7[1],x8[1]]

#=========================================================================================================================


def lost_condition(x,y,x_skew,y_skew,coll_x,coll_y,coll_s):
            return ((np.abs(x)>coll_x)|(np.abs(y)>coll_y)|(np.abs(x_skew)>coll_s)|(np.abs(y_skew)>coll_s))



def condition_in_ring(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    return ((_out_ROI_min) & (~_out_ROI_max))



def make_ROI(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2,force_normalisation = False):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    
    ROI = ((_out_ROI_min) & (~_out_ROI_max))
    # Splitting in 3 planes
    ROI_x = ROI&((np.abs(x1)>collx1)|(np.abs(x2)>collx1))
    ROI_y = ROI&((np.abs(y1)>colly1)|(np.abs(y2)>colly1))
    ROI_s = ROI&((np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    if force_normalisation:
        ROI_s = ROI&(~ROI_x)&(~ROI_y)
    # print('lol')
    return ROI,ROI_x,ROI_y,ROI_s

def make_collimation_fig(data,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                    active_drag     = "box_zoom",
                    active_scroll   = "wheel_zoom",
                    toolbar_location= "right")


    # Saving tools to tags
    # _palette = bkpalettes.Viridis8
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    # fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
    fig.tags[0]['HoverTool'].update(tooltips = [('Collimator [sigma_coll]','@opening'),('Count', '@{counts:active}')])

    # Putting legend outside
    # fig.add_layout(bkmod.Legend(), 'right')
    #=====================================

    coll_opening = 10
    coll_alpha = np.deg2rad(127.5)
    pipe_r     = cst.LHC_W_BEAM_SCREEN/2
    coll_df    = pd.DataFrame({'name'   :  ['H_left','H_right','V_top','V_bottom','S_top','S_bottom'],
                               'width'  :  pipe_r*np.ones(6),
                               'height' :2*pipe_r*np.ones(6),
                               'xs'     :6*[np.zeros(4)],
                               'ys'     :6*[np.zeros(4)],
                                'angle' :[0,0,np.pi/2,np.pi/2,coll_alpha,coll_alpha],
                                'sigma' :[data.sig_x_coll,data.sig_x_coll,data.sig_y_coll,data.sig_y_coll,data.sig_skew_coll,data.sig_skew_coll]})

        
    coll_df = update_coll(coll_df,coll_opening=coll_opening)
    source  = bkmod.ColumnDataSource(coll_df)
    
    fig.patches(xs='xs', ys='ys',alpha=1,color='gray',source=source)


    # Crop at beam pipe
    _x_pipe = np.array([-2*pipe_r] + list(np.linspace(-pipe_r,pipe_r,200)) + [2*pipe_r])
    _y_pipe = np.sqrt(pipe_r**2 - _x_pipe**2)
    _y_pipe[np.abs(_x_pipe)>pipe_r] = 0
    fig.varea(x=_x_pipe,y1=2*pipe_r*np.ones(len(_x_pipe)),y2=_y_pipe  ,color='white',alpha=1)
    fig.varea(x=_x_pipe,y2=-2*pipe_r*np.ones(len(_x_pipe)),y1=-_y_pipe,color='white',alpha=1)


        # Creating Hextiles
    #=====================================
    coll_values = np.linspace(0,10,50)
    n_bins      = 300
    coll_sig = np.max([data.sig_x_coll,data.sig_y_coll])
    XX,YY    = np.meshgrid(np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,n_bins),
                            np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,n_bins))

    _size        = np.min(np.abs(np.diff(XX.flatten())))
    _orientation = 'pointytop'

    hextiles = bkhex.hexbin(XX.flatten(), YY.flatten(), _size)
    _x,_y    = bkhex.axial_to_cartesian(hextiles.q,hextiles.r,size=_size,orientation=_orientation)
    theta_unskew= -np.deg2rad(127.5)
    _x_skew       = _x*np.cos(theta_unskew) - _y*np.sin(theta_unskew)
    _y_skew       = _x*np.cos(-theta_unskew) - _y*np.sin(-theta_unskew)
    hextiles.insert(0,'x',_x)
    hextiles.insert(1,'y',_y)
    hextiles.insert(2,'x_skew',_x_skew)
    hextiles.insert(3,'y_skew',_y_skew)
    hextiles.insert(4,'opening',np.nan)
    hextiles.counts = np.nan
    hextiles.rename(columns={'counts':'counts:active'},inplace=True)
    #=====================================



    _sig_x    = data.sig_x_coll
    _sig_y    = data.sig_y_coll
    _sig_skew = data.sig_skew_coll
    # Looping over chunks

    name  = 0
    group = data.data.groupby('Chunk ID').get_group(name)
    hextiles.insert(len(hextiles.columns),f'counts:{name}',np.nan)

    total = 0
    for coll_min,coll_max in zip(coll_values[:-1],coll_values[1:]):
        
        # Identifying Hex in ROI
        
        hex_,hex_x,hex_y,hex_s  = make_ROI( hextiles['x'],hextiles['x'],
                                        hextiles['y'],hextiles['y'],
                                        hextiles['x_skew'],hextiles['y_skew'],
                                        coll_min*_sig_x,coll_max*_sig_x,
                                        coll_min*_sig_y,coll_max*_sig_y,
                                        coll_min*_sig_skew,coll_max*_sig_skew,force_normalisation=True)
        hextiles.loc[(hex_x|hex_y|hex_s),f'opening'] = np.mean([coll_min,coll_max])


        # Counts per collimators
        #------------------------------------
        count_,count_x,count_y,count_s = make_ROI( group['x_max'],group['x_min'],
                                            group['y_max'],group['y_min'],
                                            group['skew_max'],group['skew_min'],
                                            coll_min*_sig_x,coll_max*_sig_x,
                                            coll_min*_sig_y,coll_max*_sig_y,
                                            coll_min*_sig_skew,coll_max*_sig_skew)

        
        # Initializing counts
        #------------------------------------
        if count_.sum() != 0:
            hextiles.loc[(hex_x|hex_y|hex_s),f'counts:{name}'] = 0

        # Show plane by plane:
        #------------------------------------
        hextiles.loc[hex_x,f'counts:{name}'] = (count_x&~count_y&~count_s).sum()
        hextiles.loc[hex_y,f'counts:{name}'] = (~count_x&count_y&~count_s).sum()
        hextiles.loc[hex_s,f'counts:{name}'] = (~count_x&~count_y&count_s).sum()

        # Add cross planes:
        #------------------------------------
        hextiles.loc[hex_x,f'counts:{name}'] += (count_x&count_y&~count_s).sum()
        hextiles.loc[hex_y,f'counts:{name}'] += (count_x&count_y&~count_s).sum()

        hextiles.loc[hex_x,f'counts:{name}'] += (count_x&~count_y&count_s).sum()
        hextiles.loc[hex_s,f'counts:{name}'] += (count_x&~count_y&count_s).sum()

        hextiles.loc[hex_y,f'counts:{name}'] += (~count_x&count_y&count_s).sum()
        hextiles.loc[hex_s,f'counts:{name}'] += (~count_x&count_y&count_s).sum()

        # Cleaning counts
        #------------------------------------
        if count_.sum() == 0:
            hextiles.loc[(hex_x|hex_y|hex_s),f'counts:{name}'] = np.nan


        # # Show total
        # #------------------------------------
        # hextiles.loc[(hex_x|hex_y|hex_s),f'counts:{name}'] = count_x.sum()+count_y.sum()+count_s.sum()
        # total += count_s.sum()
        # print(total)

    # hextiles[hextiles[f'counts:{name}'] == 0] = np.nan
    hextiles['counts:active'] = hextiles['counts:0']

    
    # hextiles.dropna(inplace=True)
    source    = bkmod.ColumnDataSource(hextiles)
    nan_color = bkcolors.RGB(255,255,255,a=0)
    cmap      = bktrfm.linear_cmap('counts:active', 'Plasma256', 0, 1500,nan_color=nan_color)
    fig.hex_tile(q="q", r="r", size= _size, line_color=None, source=source,alpha=1,fill_color=cmap)


    # color_bar = r.construct_color_bar(padding=0,
    #                                   ticker=p.xaxis.ticker,
    #                                   formatter=p.xaxis.formatter)

    color_bar = bkmod.ColorBar(title='Counts',color_mapper=cmap['transform'])
    fig.add_layout(color_bar, 'right')


    # Axis and Legend
    #=====================================

    fig.xaxis.axis_label = 'x [m]'
    fig.yaxis.axis_label = 'y [m]'
    fig.legend.title     = r'Particles ID'
    fig.legend.click_policy="hide"
    # fig.x_range=bkmod.Range1d(-6, 6)
    # fig.y_range=bkmod.Range1d(-6, 6)
    # fig.match_aspect=True

    #=====================================

    return fig

fig = make_collimation_fig(data,title='Maximum excursion map',width=1000,height=700)
# fig.aspect_ratio = 1
bktools.set_aspect(fig , x_lim=(-3e-3,3e-3),y_lim=(-3e-3,3e-3), aspect=0.9)
bk.show(fig)

In [ ]:


def update_coll(_df,coll_opening = 5):
    
    # Resetting values
    #-----------------------------
    _width  = _df['width'].unique()[0]
    _height = _df['height'].unique()[0]
    _df['xs']   = 6*[np.array([0,0,_width,_width])]
    _df['ys']   = 6*[np.array([-_height/2,_height/2,_height/2,-_height/2])]

    # Distributing colls on x-axis
    #-----------------------------
    _df['xs'] += coll_opening*_df['sigma']
    _df.loc[_df.name.str.contains('left|bottom'),'xs']   *= -1

    # Rotating according to angle
    _x,_y  = np.stack(_df['xs']),np.stack(_df['ys'])
    _angle = np.stack(_df['angle'].apply(lambda angle: list(np.repeat(angle,4))))
    _x_rot = _x*np.cos(_angle) - _y*np.sin(_angle)
    _y_rot = _x*np.sin(_angle) + _y*np.cos(_angle)


    _df['xs'] = list(_x_rot)
    _df['ys'] = list(_y_rot)

    # Returning in mm
    # _df['xs'] *= 
    # _df['ys'] *= 

    return _df


def excursion_polygon(row):
    # skew col : y = ax + b
    skew_angle = 127.5
    a = np.tan(np.deg2rad(skew_angle-90))
    b = np.max([np.abs(row['skew_max']),np.abs(row['skew_min'])])/np.cos(np.deg2rad(skew_angle-90))

    # skew col : y = ax - b, x = (y+b)/a
    x1 = [row['x_max'],a*row['x_max'] - b]
    x2 = [(row['y_min']+b)/a,row['y_min']]

    # skew col : y = -ax - b, x = -(y+b)/a
    x3 = [-(row['y_min']+b)/a,row['y_min']]
    x4 = [row['x_min'],-a*row['x_min'] - b]

    # skew col : y = ax + b, x = (y-b)/a
    x5 = [row['x_min'],a*row['x_min'] + b]
    x6 = [(row['y_max']-b)/a,row['y_max']]

    # skew col : y = -ax + b, x = -(y-b)/a
    x7 = [-(row['y_max']-b)/a,row['y_max']]
    x8 = [row['x_max'],-a*row['x_max'] + b]

    if x5[1]<x4[1]:
        x4[1],x5[1] = x5[1],x4[1]
    
    if x8[1]<x1[1]:
        x1[1],x8[1] = x8[1],x1[1]

    return [x1[0],x2[0],x3[0],x4[0],x5[0],x6[0],x7[0],x8[0]], [x1[1],x2[1],x3[1],x4[1],x5[1],x6[1],x7[1],x8[1]]

#=========================================================================================================================


def lost_condition(x,y,x_skew,y_skew,coll_x,coll_y,coll_s):
            return ((np.abs(x)>coll_x)|(np.abs(y)>coll_y)|(np.abs(x_skew)>coll_s)|(np.abs(y_skew)>coll_s))



def condition_in_ring(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    return ((_out_ROI_min) & (~_out_ROI_max))



def make_ROI(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    
    ROI = ((_out_ROI_min) & (~_out_ROI_max))
    # Splitting in 3 planes
    ROI_x = ROI&((np.abs(x1)>collx1)|(np.abs(x2)>collx1))
    ROI_y = ROI&((np.abs(y1)>colly1)|(np.abs(y2)>colly1))
    ROI_s = ROI&(~ROI_x)&(~ROI_y)


    return ROI_x,ROI_y,ROI_s

def plane_condition(x1,x2,collx1,collx2):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1)) 
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2))
    
    ROI = ((_out_ROI_min) & (~_out_ROI_max))

    return ROI

def make_collimation_fig(data,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                    active_drag     = "box_zoom",
                    active_scroll   = "wheel_zoom",
                    toolbar_location= "right")


    # Saving tools to tags
    # _palette = bkpalettes.Viridis8
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    # fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
    fig.tags[0]['HoverTool'].update(tooltips = [('Collimator [sigma_coll]','@opening'),('Count', '@{counts:active}')])

    # Putting legend outside
    # fig.add_layout(bkmod.Legend(), 'right')
    #=====================================

    coll_opening = 10
    coll_alpha = np.deg2rad(127.5)
    pipe_r     = cst.LHC_W_BEAM_SCREEN/2
    coll_df    = pd.DataFrame({'name'   :  ['H_left','H_right','V_top','V_bottom','S_top','S_bottom'],
                               'width'  :  pipe_r*np.ones(6),
                               'height' :2*pipe_r*np.ones(6),
                               'xs'     :6*[np.zeros(4)],
                               'ys'     :6*[np.zeros(4)],
                                'angle' :[0,0,np.pi/2,np.pi/2,coll_alpha,coll_alpha],
                                'sigma' :[data.sig_x_coll,data.sig_x_coll,data.sig_y_coll,data.sig_y_coll,data.sig_skew_coll,data.sig_skew_coll]})

        
    coll_df = update_coll(coll_df,coll_opening=coll_opening)
    source  = bkmod.ColumnDataSource(coll_df)
    
    fig.patches(xs='xs', ys='ys',alpha=1,color='gray',source=source)


    # Crop at beam pipe
    _x_pipe = np.array([-2*pipe_r] + list(np.linspace(-pipe_r,pipe_r,200)) + [2*pipe_r])
    _y_pipe = np.sqrt(pipe_r**2 - _x_pipe**2)
    _y_pipe[np.abs(_x_pipe)>pipe_r] = 0
    fig.varea(x=_x_pipe,y1=2*pipe_r*np.ones(len(_x_pipe)),y2=_y_pipe  ,color='white',alpha=1)
    fig.varea(x=_x_pipe,y2=-2*pipe_r*np.ones(len(_x_pipe)),y1=-_y_pipe,color='white',alpha=1)


        # Creating Hextiles
    #=====================================
    coll_values = np.linspace(0,10,50)
    n_bins      = 300
    coll_sig = np.max([data.sig_x_coll,data.sig_y_coll])
    XX,YY    = np.meshgrid(np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,n_bins),
                            np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,n_bins))

    _size        = np.min(np.abs(np.diff(XX.flatten())))
    _orientation = 'pointytop'

    hextiles = bkhex.hexbin(XX.flatten(), YY.flatten(), _size)
    _x,_y    = bkhex.axial_to_cartesian(hextiles.q,hextiles.r,size=_size,orientation=_orientation)
    theta_unskew= -np.deg2rad(127.5)
    _x_skew       = _x*np.cos(theta_unskew) - _y*np.sin(theta_unskew)
    _y_skew       = _x*np.cos(-theta_unskew) - _y*np.sin(-theta_unskew)
    hextiles.insert(0,'x',_x)
    hextiles.insert(1,'y',_y)
    hextiles.insert(2,'x_skew',_x_skew)
    hextiles.insert(3,'y_skew',_y_skew)
    hextiles.insert(4,'opening',np.nan)
    hextiles.counts = np.nan
    hextiles.rename(columns={'counts':'counts:active'},inplace=True)
    #=====================================



    _sig_x    = data.sig_x_coll
    _sig_y    = data.sig_y_coll
    _sig_skew = data.sig_skew_coll
    # Looping over chunks

    # name  = 0
    # group = data.data.groupby('Chunk ID').get_group(name)
    # group = group.set_index('particle').loc[[10]]
    

    for name,group in data.data.groupby('Chunk ID'):
        hextiles.insert(len(hextiles.columns),f'counts:{name}',np.nan)
        for coll_min,coll_max in zip(coll_values[:-1],coll_values[1:]):
            
            # Identifying Hex in ROI
            
            hex_x,hex_y,hex_s  = make_ROI( hextiles['x'],hextiles['x'],
                                            hextiles['y'],hextiles['y'],
                                            hextiles['x_skew'],hextiles['y_skew'],
                                            coll_min*_sig_x,coll_max*_sig_x,
                                            coll_min*_sig_y,coll_max*_sig_y,
                                            coll_min*_sig_skew,coll_max*_sig_skew)
            hextiles.loc[(hex_x|hex_y|hex_s),f'opening'] = np.mean([coll_min,coll_max])


            # Counts per collimators
            #------------------------------------
            count_x = plane_condition(group['x_min'],group['x_max'],coll_min*_sig_x,coll_max*_sig_x)
            count_y = plane_condition(group['y_min'],group['y_max'],coll_min*_sig_y,coll_max*_sig_y)
            count_s = plane_condition(group['skew_min'],group['skew_max'],coll_min*_sig_skew,coll_max*_sig_skew)
            
            # Initializing counts
            #------------------------------------
            if count_x.sum()+count_y.sum()+count_s.sum() != 0:
                hextiles.loc[(hex_x|hex_y|hex_s),f'counts:{name}'] = 0

            # Show plane by plane:
            #------------------------------------
            hextiles.loc[hex_x,f'counts:{name}'] = count_x.sum()
            hextiles.loc[hex_y,f'counts:{name}'] = count_y.sum()
            hextiles.loc[hex_s,f'counts:{name}'] = count_s.sum()

            # Cleaning counts
            #------------------------------------
            if count_x.sum()+count_y.sum()+count_s.sum() == 0:
                hextiles.loc[(hex_x|hex_y|hex_s),f'counts:{name}'] = np.nan


    hextiles['counts:active'] = hextiles['counts:0']

    data_col  = [col for col in hextiles.columns if 'counts:' in col]
    max_value = np.max(hextiles[data_col].sum(axis=1)) 
    source    = bkmod.ColumnDataSource(hextiles[['q','r','opening']+data_col])
    nan_color = bkcolors.RGB(255,255,255,a=0)
    # cmap      = bktrfm.linear_cmap('counts:active', 'Plasma256', 0, 1500,nan_color=nan_color)
    cmap      = bktrfm.log_cmap('counts:active', 'Magma256', 1, max_value,nan_color=nan_color)
    fig.hex_tile(q="q", r="r", size= _size, line_color=None, source=source,alpha=1,fill_color=cmap)


    color_bar = bkmod.ColorBar(title='Counts',color_mapper=cmap['transform'])
    fig.add_layout(color_bar, 'right')


    # Axis and Legend
    #=====================================

    fig.xaxis.axis_label = 'x [m]'
    fig.yaxis.axis_label = 'y [m]'
    # fig.legend.title     = r'Particles ID'
    # fig.legend.click_policy="hide"
    # fig.x_range=bkmod.Range1d(-6, 6)
    # fig.y_range=bkmod.Range1d(-6, 6)
    # fig.match_aspect=True

    #=====================================

    return fig

fig = make_collimation_fig(data,title='Maximum excursion map',width=1000,height=700)
# fig.aspect_ratio = 1
bktools.set_aspect(fig , x_lim=(-3e-3,3e-3),y_lim=(-3e-3,3e-3), aspect=0.9)
bk.show(fig)

In [ ]:
group.set_index('particle').loc[[0]]

In [ ]:
def lost_condition(x,y,x_skew,y_skew,coll_x,coll_y,coll_s):
            return ((np.abs(x)>coll_x)|(np.abs(y)>coll_y)|(np.abs(x_skew)>coll_s)|(np.abs(y_skew)>coll_s))


# Creating Hextiles
#=====================================
coll_values = np.linspace(0,10,50)
coll_sig = np.max([data.sig_x_coll,data.sig_y_coll])
XX,YY    = np.meshgrid(np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,300),
                        np.linspace(-coll_values[-1]*coll_sig,coll_values[-1]*coll_sig,300))

_size        = np.min(np.abs(np.diff(XX.flatten())))
_orientation = 'pointytop'

hextiles = bkhex.hexbin(XX.flatten(), YY.flatten(), _size)
_x,_y    = bkhex.axial_to_cartesian(hextiles.q,hextiles.r,size=_size,orientation=_orientation)
theta_unskew= -np.deg2rad(127.5)
_x_skew       = _x*np.cos(theta_unskew) - _y*np.sin(theta_unskew)
_y_skew       = _x*np.cos(-theta_unskew) - _y*np.sin(-theta_unskew)
hextiles.insert(0,'x',_x)
hextiles.insert(1,'y',_y)
hextiles.insert(2,'x_skew',_x_skew)
hextiles.insert(3,'y_skew',_y_skew)
hextiles.insert(4,'opening',np.nan)
hextiles.counts = np.nan
hextiles.rename(columns={'counts':'counts:active'},inplace=True)
#=====================================





def make_ROI(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2,force_normalisation = False):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    
    ROI = ((_out_ROI_min) & (~_out_ROI_max))
    # Splitting in 3 planes
    ROI_x = ROI&((np.abs(x1)>collx1)|(np.abs(x2)>collx1))
    ROI_y = ROI&((np.abs(y1)>colly1)|(np.abs(y2)>colly1))
    ROI_s = ROI&((np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    if force_normalisation:
        ROI_s = ROI&(~ROI_x)&(~ROI_y)
    # print('lol')
    return ROI,ROI_x,ROI_y,ROI_s

_sig_x    = data.sig_x_coll
_sig_y    = data.sig_y_coll
_sig_skew = data.sig_skew_coll
# Looping over chunks

name  = 22
group = data.data.groupby('Chunk ID').get_group(name)
hextiles.insert(len(hextiles.columns),f'counts:{name}',np.nan)

for coll_min,coll_max in zip(coll_values[:-1],coll_values[1:]):
    
    # Identifying Hex in ROI
    
    hex_tot,hex_x,hex_y,hex_s  = make_ROI( hextiles['x'],hextiles['x'],
                                    hextiles['y'],hextiles['y'],
                                    hextiles['x_skew'],hextiles['y_skew'],
                                    coll_min*_sig_x,coll_max*_sig_x,
                                    coll_min*_sig_y,coll_max*_sig_y,
                                    coll_min*_sig_skew,coll_max*_sig_skew)
    hextiles.loc[(hex_x|hex_y|hex_s),f'opening'] = np.mean([coll_min,coll_max])


    # Counts per collimators
    #------------------------------------
    count_tot,count_x,count_y,count_s = make_ROI( group['x_max'],group['x_min'],
                                        group['y_max'],group['y_min'],
                                        group['skew_max'],group['skew_min'],
                                        coll_min*_sig_x,coll_max*_sig_x,
                                        coll_min*_sig_y,coll_max*_sig_y,
                                        coll_min*_sig_skew,coll_max*_sig_skew)
    print(count_x.sum(),count_y.sum(),count_s.sum())
    hextiles.loc[hex_x,f'counts:{name}'] = count_x.sum()
    hextiles.loc[hex_y,f'counts:{name}'] = count_y.sum()
    hextiles.loc[hex_s,f'counts:{name}'] = count_s.sum()

    # if count_x.sum()>10:
    #       break

# hextiles['counts:active'] = hextiles['counts:0']
      



# for name,group in J_df.groupby('Chunk ID'):
#         # Forcing corner values to have same grid.
#         _hex = bkhex.hexbin(np.array(list(group['Jx/emitt']) + [J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max()]),
#                             np.array(list(group['Jy/emitt']) + [J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max()]),
#                             size=_size,orientation=_orientation)

#         # Removing corner values
#         _hex = _hex[1:-1]

#         # Adding chunk ID
#         hextiles_template.insert(name+1,f'counts:{name}',_hex.set_index(['q','r'])['counts'])



# hextiles_template.reset_index(inplace=True)
# hextiles_template['counts:active'] = hextiles_template['counts:0']


# coll_values = np.linspace(0,10,50)
# turn_list   = list(data.data.groupby('start_at_turn').groups.keys())
# selected_turn = turn_list[16]
# for coll_min,coll_max in zip(coll_values[:-1],coll_values[1:]):

#     try:
#         counts = (data.compute_intensity(coll_opening=coll_max,at_turn = selected_turn)['count']- data.compute_intensity(coll_opening=coll_min,at_turn = selected_turn)['count']).values[-1]
#     except:
#         counts = np.nan
    
#     _out_ROI_max = lost_condition(hextiles['x'],hextiles['y'],hextiles['x_skew'],hextiles['y_skew'],coll_max*data.sig_x_coll,coll_max*data.sig_y_coll,coll_max*data.sig_skew_coll)
#     _out_ROI_min = lost_condition(hextiles['x'],hextiles['y'],hextiles['x_skew'],hextiles['y_skew'],coll_min*data.sig_x_coll,coll_min*data.sig_y_coll,coll_min*data.sig_skew_coll)
#     _out_ROI = ((_out_ROI_min) & (~_out_ROI_max))
# #         
#     hextiles.loc[_out_ROI,'counts'] = counts
#     hextiles.loc[_out_ROI,'opening'] = np.mean([coll_min,coll_max])

In [ ]:

def make_ROI(x1,x2,y1,y2,s1,s2,collx1,collx2,colly1,colly2,colls1,colls2,force_normalisation = False):
    _out_ROI_min  =((np.abs(x1)>collx1)|(np.abs(x2)>collx1) | 
                    (np.abs(y1)>colly1)|(np.abs(y2)>colly1) | 
                    (np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    _out_ROI_max  =((np.abs(x1)>collx2)|(np.abs(x2)>collx2) | 
                    (np.abs(y1)>colly2)|(np.abs(y2)>colly2) | 
                    (np.abs(s1)>colls2)|(np.abs(s2)>colls2))
    
    ROI = ((_out_ROI_min) & (~_out_ROI_max))
    # Splitting in 3 planes
    ROI_x = ROI&((np.abs(x1)>collx1)|(np.abs(x2)>collx1))
    ROI_y = ROI&((np.abs(y1)>colly1)|(np.abs(y2)>colly1))
    ROI_s = ROI&((np.abs(s1)>colls1)|(np.abs(s2)>colls1))
    if force_normalisation:
        ROI_s = ROI&(~ROI_x)&(~ROI_y)
    # print('lol')
    return ROI,ROI_x,ROI_y,ROI_s

    
total = 0
for coll_min,coll_max in zip(coll_values[:-1],coll_values[1:]):
    # y_cond_min = (np.abs(group['y_max'])>coll_min*data.sig_y_coll)|(np.abs(group['y_min'])>coll_min*data.sig_y_coll)
    # y_cond_max = (np.abs(group['y_max'])>coll_max*data.sig_y_coll)|(np.abs(group['y_min'])>coll_max*data.sig_y_coll)
    # y_ring = y_cond_min&(~y_cond_max)
    # total += y_ring.sum()
    # print(y_cond_min.sum(),y_cond_max.sum(),y_ring.sum(),total)


    

    count_tot,count_x,count_y,count_s = make_ROI( group['x_max'],group['x_min'],
                                        group['y_max'],group['y_min'],
                                        group['skew_max'],group['skew_min'],
                                        coll_min*_sig_x,coll_max*_sig_x,
                                        coll_min*_sig_y,coll_max*_sig_y,
                                        coll_min*_sig_skew,coll_max*_sig_skew)
    total += count_y.sum()
    print(count_y.sum(),total)


In [ ]:
sum(count_tot),sum(count_x),sum(count_y),sum(count_s),sum(count_x)+sum(count_y)+sum(count_s)

In [ ]:
coll_min = 3
coll_max = 3.1
make_ROI( group['x_max'],group['x_min'],
                                        group['y_max'],group['y_min'],
                                        group['skew_max'],group['skew_min'],
                                        coll_min*_sig_x,coll_max*_sig_x,
                                        coll_min*_sig_y,coll_max*_sig_y,
                                        coll_min*_sig_skew,coll_max*_sig_skew)

In [ ]:
np.sum(count_x)

In [ ]:
count_x.sum()

In [ ]:
intensity_df = {}
coll_values = np.linspace(0,10,50)
for coll_val in  coll_values:
    try:
        intensity_df[coll_val] = data.compute_intensity(coll_opening=coll_val).set_index('Chunk ID')[['count','survived']][1:]
    except:
        intensity_df[coll_val] = None

In [ ]:
intensity_df = {}
coll_values = np.linspace(0,10,50)
for coll_val in  coll_values:
    try:
        intensity_df[coll_val] = data.compute_intensity(coll_opening=coll_val,find_plane=True).set_index('Chunk ID')[['count','survived']][1:]
    except:
        intensity_df[coll_val] = None

In [ ]:
(data.compute_intensity(coll_opening=coll_max).set_index('Chunk ID')['count']- data.compute_intensity(coll_opening=coll_min,at_turn = selected_turn)['count']).values[-1]

In [ ]:
self = data
coll_opening = 5
from_df='_data'
at_turn = None


_sigx = np.sqrt(self.betx*3.5e-6/self.particle_on_co.gamma0[0])
_sigy = np.sqrt(self.bety*3.5e-6/self.particle_on_co.gamma0[0])
# Ellipse in polar: r(alpha) = sqrt((a*cos(alpha))^2 + (b*sin(alpha))^2)
_alpha   = np.deg2rad(127.5)
_sigskew = np.sqrt((_sigx*np.cos(_alpha))**2 + (_sigy*np.sin(_alpha))**2)

# Collimator opening
coll_x = coll_opening*_sigx
coll_y = coll_opening*_sigy
coll_s = coll_opening*_sigskew



def plane_lost(df):
    _plane  = pd.Series('',index=df.x_min.index)
    idx_x   = _plane.index[(np.abs(df.x_min)>coll_x)|(np.abs(df.x_max)>coll_x)]
    idx_y   = _plane.index[(np.abs(df.y_min)>coll_y)|(np.abs(df.y_max)>coll_y)]
    idx_skew= _plane.index[(np.abs(df.skew_min)>coll_s)|(np.abs(df.skew_max)>coll_s)]

    _plane.loc[idx_x] += 'x'
    _plane.loc[idx_y] += 'y'
    _plane.loc[idx_skew] += 's'

    return _plane


# Keep columns
coordinates = ['x','y','skew']
keep_cols   = [f'{i}_min' for i in coordinates] + [f'{i}_max' for i in coordinates]
keep_cols   = ['Chunk ID','particle','start_at_turn','stop_at_turn'] + keep_cols

if from_df == '_data':
    group  = self.data[keep_cols]
    if at_turn is not None:
        group  = group[group.start_at_turn <= at_turn]
elif from_df == '_checkpoint':
    #TODO
    pass
elif from_df == '_df':
    #TODO
    pass


_plane_lost  = plane_lost(group)
_lost        = _plane_lost.apply(lambda plane_str: len(plane_str)>0)
idx_lost     = group.index[_lost]
idx_survived = group.index[~_lost]

# New columns

group.insert(0,'beyond_coll',False)
group.insert(0,'lost',False)


# Finding lost particles
group.loc[idx_lost,'beyond_coll'] = True
group.loc[:,'lost'] = group.groupby('particle').beyond_coll.cumsum().astype(bool)


# Finding lost plane:
group.insert(0,'plane',_plane_lost)
group.loc[_lost,'plane'] += '|'
_plane_df = group[['particle','plane']]
_plane_result = _plane_df.groupby('particle')['plane'].apply(pd.Series.cumsum).apply(lambda _str: _str.split('|')[0]).to_frame()
_plane_result.insert(0,'index',_plane_result.index.get_level_values(1))
_plane_result = _plane_result.sort_values('index').set_index('index')
group.loc[:,'plane'] = _plane_result['plane']

intensity = group[~group.lost].groupby('start_at_turn').count().particle
intensity = group[~group.lost].groupby('start_at_turn').count().particle.to_frame()
intensity.insert(0,'stop_at_turn',group.groupby('start_at_turn').stop_at_turn.max())
intensity.insert(1,'Chunk ID',group.groupby('start_at_turn')['Chunk ID'].max())
intensity.reset_index(drop=False,inplace=True)
intensity.rename(columns={'particle':'count'},inplace=True)


survived  = group[~group.lost].groupby('start_at_turn').apply(lambda group: list(group.particle.values))
lost      = group[group.lost].groupby('start_at_turn').apply(lambda group: list(group.particle.values))
plane     = group[group.lost].groupby('start_at_turn').apply(lambda group: list(group.plane.values))
intensity.insert(3,'survived',survived.values)
intensity.insert(4,'lost',lost.values)
intensity.insert(5,'plane',plane.values)

starting_point = pd.DataFrame({'Chunk ID':[-1],'start_at_turn':[-1],'stop_at_turn':[0],'count':[len(group.particle.unique())],'survived':[list(group.particle.unique())]})
intensity      = pd.concat([starting_point,intensity]).reset_index(drop=True)
intensity

In [ ]:
2279+17721

In [ ]:
pd.Series(intensity.loc[23]['plane']).value_counts()

In [ ]:
lost      = group[group.lost].groupby('start_at_turn').apply(lambda group: list(group.particle.values))
plane     = group[group.lost].groupby('start_at_turn').apply(lambda group: list(group.plane.values))

In [ ]:
lost

In [ ]:
plane

In [ ]:
group.groupby('particle')['plane'].astype(str)

In [ ]:
_plane

In [ ]:
group.groupby('particle')['plane'].apply(pd.Series.cumsum).apply(lambda _str: _str.split('|')[0])

In [ ]:
group.set_index(['particle','start_at_turn']).index

In [ ]:
group.groupby('particle').beyond_coll.cumsum()#.astype(bool)

In [ ]:
np.cumsum(['1','2','3'])

In [ ]:
group.groupby('particle').apply(lambda part: sum(part['plane']),axis=1)

In [ ]:
intensity

In [ ]:
intensity

In [ ]:
_test.loc[_lost]

In [ ]:
group.x_min

pd.Series(np.nan,index=group.x_min.index).index

In [ ]:
group.x_min>coll_x

In [ ]:
x = np.array([1,1])

In [ ]:
group.x_max.copy()*0

In [ ]:
intensity['lost'].apply(lambda x: len(x))

In [ ]:
data.compute_intensity(coll_opening=5)

In [ ]:
data.data.groupby('start_at_turn')['Chunk ID'].max()

In [ ]:
data.checkpoint[:2]

In [ ]:
(data.checkpoint_sig[:2][['x_sig']]**2+data.checkpoint_sig[:2][['px_sig']]**2)*data.sig_x

x/sig_x

In [ ]:

def make_JxJy_fig(data,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                    active_drag     = "pan",
                    active_scroll   = "wheel_zoom",
                    toolbar_location= "right")


    # Saving tools to tags
    # _palette = bkpalettes.Viridis8
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    # fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
    fig.tags[0]['HoverTool'].update(tooltips = [('Collimator [sigma_coll]','@opening'),('Count', '@counts')])

    # Putting legend outside
    # fig.add_layout(bkmod.Legend(), 'right')
    #=====================================

    J_df = data.checkpoint.groupby('Chunk ID').get_group(1)[['BUNCH','Chunk ID','turn','particle']]
    J_df.insert(4,'Jx/emitt',1/2 * (data.checkpoint_sig.x_sig**2 + data.checkpoint_sig.px_sig**2))
    J_df.insert(5,'Jy/emitt',1/2 * (data.checkpoint_sig.y_sig**2 + data.checkpoint_sig.py_sig**2))
    J_df.dropna(inplace=True)

  
    # coll_sig = np.max([data.sig_x_coll,data.sig_y_coll])
    coll_max = 10
    _x_dummy = np.linspace(0,10**2/2 ,300)


    _size        = np.min(np.abs(np.diff(_x_dummy)))
    _orientation = 'pointytop'

    hextiles = bkhex.hexbin(J_df['Jx/emitt'],J_df['Jy/emitt'], _size)



    source = hextiles
    cmap   = bktrfm.linear_cmap('counts', 'Viridis256', 0, max(source.counts))
    fig.hex_tile(q="q", r="r", size= _size, line_color=None, source=source,alpha=1,fill_color=cmap)

    fig.hspan(y=[0], line_width=[2], line_color="black")
    fig.vspan(x=[0], line_width=[2], line_color="black")

    # source = bkmod.ColumnDataSource(J_df)
    # fig.scatter('Jx/emitt','Jy/emitt', alpha=0.6, source=source)


    color_bar = bkmod.ColorBar(title='Counts',color_mapper=cmap['transform'])
    fig.add_layout(color_bar, 'right')


    # Axis and Legend
    #=====================================

    fig.xaxis.axis_label = 'Jx/emitt'
    fig.yaxis.axis_label = 'Jy/emitt'


    #=====================================

    return fig

fig = make_JxJy_fig(data,title='(Jx,Jy) Distribution',width=1000,height=700)
# fig.aspect_ratio = 1
bktools.set_aspect(fig , x_lim=(-5,65),y_lim=(-5,50), aspect=0.9)
bk.show(fig)

In [ ]:
data.n_parts//

In [ ]:
def test_slider(data,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,hover,wheel_zoom",
                    active_drag     = "pan",
                    active_scroll   = "wheel_zoom",
                    toolbar_location= "right")


    # Saving tools to tags
    # _palette = bkpalettes.Viridis8
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    # fig.tags[0]['WheelZoomTool'].update(dimensions = 'height')
    # fig.tags[0]['HoverTool'].update(tooltips = [('Variable', '$name'),('s [m]','$x{0}'),(f'Value', '$y'),('Element','@name')])
    fig.tags[0]['HoverTool'].update(tooltips = [('Collimator [sigma_coll]','@opening'),('Count', '@counts')])

    # Putting legend outside
    # fig.add_layout(bkmod.Legend(), 'right')
    #=====================================


    chunk_df = data.data[['Chunk ID','start_at_turn','stop_at_turn']].groupby('Chunk ID').mean()
    chunk_df.insert(0,'x',(chunk_df['stop_at_turn']+chunk_df['start_at_turn'])/2)

    to_source = bktools.source_from_groupby(chunk_df,by='Chunk ID',columns = ['x'])
    # to_source
    
    to_source.insert(0,'width',(chunk_df['stop_at_turn']-chunk_df['start_at_turn']).max())
    to_source.insert(1,'height',1)
    to_source.insert(2,'y',0.5)

        
    # source = bkto_source
    source_chunk  = bkmod.ColumnDataSource(to_source)
    
    fig.rect(x='x:active', y='y', width='width', height='height',alpha=0.5,source=source_chunk)


    for coll_opening,color in zip([3,4,5,6,7,8,9,10][::-1],fig.tags[1]['palette']):
        # Creating source
        #=====================================
        intensity_df = data.compute_intensity(coll_opening=coll_opening)
        intensity_df = intensity_df[1:]
        intensity_df.insert(3,'Norm. Count',np.abs(intensity_df['count'])/intensity_df.loc[1,'count'])
        source       = bkmod.ColumnDataSource(intensity_df[['start_at_turn','Norm. Count']])
        #=====================================


        # Plotting
        #=====================================
        legend_opening = str(coll_opening).ljust(4-len(str(coll_opening)))
        fig.step(x='start_at_turn',y='Norm. Count', source=source,legend_label=f'{legend_opening} σ_coll, [I(0) = {str(intensity_df.loc[1,"count"]).ljust(7)} p+]',line_width=2,color=color)
        #=====================================




    




    padding =  100
    chunk_slider = bkmod.Slider(start=data.data['Chunk ID'].min(), end=data.data['Chunk ID'].max(), value=0, step=1, title="Chunk ID",width = width-2*padding,margin=[0,100])
    fig.min_border_right = padding
    fig.min_border_left  = padding


    callback = bkmod.callbacks.CustomJS(args=dict(slider = chunk_slider,source = source_chunk), code="""
                //=========================================================
                source.data['x:active'] = source.data['x:'+slider.value.toString()];
                source.change.emit()
                //=========================================================""")

    chunk_slider.js_on_change('value', callback)


    return fig,chunk_slider



fig,chunk_slider= test_slider(data,title='(Jx,Jy) Distribution',width=1000,height=700)
# fig.aspect_ratio = 1
# bktools.set_aspect(fig , x_lim=(-5,65),y_lim=(-5,50), aspect=0.9)
bk.show(bklay.column(fig,chunk_slider))

In [ ]:
chunk_slider = bkmod.Slider(start=data.data['Chunk ID'].min(), end=data.data['Chunk ID'].max(), value=0, step=1, title="Chunk ID",width = 100,margin=[0,100])


def make_phasespace_fig(df,xy,slider,title=None,width=2000,height=400):

    # Creating Figure
    #=====================================
    fig = bk.figure(output_backend  = "webgl",
                    height          = height, 
                    width           = width,
                    title           = title, 
                    tools           = "box_zoom,pan,reset,save,hover,wheel_zoom,crosshair",
                    active_drag     = "box_zoom",
                    active_scroll   = "wheel_zoom",
                    active_inspect  = None,
                    toolbar_location= "right")


    # Saving tools to tags
    _palette = bkpalettes.Spectral10
    fig.tags = [{str(type(t)).split('.')[-1].split('\'')[0]:t for t in fig.tools},
                {'palette':_palette}]
    fig.tags[0]['HoverTool'].update(tooltips = [('Particle', '$index'),(f'Coordinates', '($x,$y)')])
    #=====================================


    # Creating source
    #=====================================
    x,y = xy
    to_source = bktools.source_from_groupby(df,by='Chunk ID',columns = [x,y])
    to_source.insert(0,'particle',df.groupby('Chunk ID').get_group(0).particle)
    source = bkmod.ColumnDataSource(to_source)

    #=====================================


    # Plotting
    #=====================================
    fig.scatter(f'{x}:active',f'{y}:active', alpha=0.3, source=source)
    #=====================================

    # Adding slider callback
    #=====================================

    callback = bkmod.callbacks.CustomJS(args=dict(slider = slider,source = source), code=f"""
                //=========================================================
                source.data['{x}:active'] = source.data['{x}:'+slider.value.toString()];
                source.data['{y}:active'] = source.data['{y}:'+slider.value.toString()];
                source.change.emit()
                //=========================================================""")

    slider.js_on_change('value', callback)
    #=====================================


    # Axis and Legend
    #=====================================

    fig.xaxis.axis_label = x
    fig.yaxis.axis_label = y


    #=====================================

    return fig



BOKEH_FIGS['x-px'] = make_phasespace_fig(data.checkpoint_sig,xy=('x_sig','px_sig'),slider=chunk_slider,title='x norm. phase space',width=_default_fig_width//4,height=_default_fig_height)
BOKEH_FIGS['y-py'] = make_phasespace_fig(data.checkpoint_sig,xy=('y_sig','py_sig'),slider=chunk_slider,title='y norm. phase space',width=_default_fig_width//4,height=_default_fig_height)
BOKEH_FIGS['zeta-pzeta'] = make_phasespace_fig(data.checkpoint_sig,xy=('zeta_sig','pzeta_sig'),slider=chunk_slider,title='zeta norm. phase space',width=_default_fig_width//4,height=_default_fig_height)

BOKEH_FIGS['x-y'] = make_phasespace_fig(data.checkpoint_sig,xy=('x_sig','y_sig'),slider=chunk_slider,title='Transverse norm space',width=_default_fig_width//4,height=_default_fig_height)


bktools.set_aspect(BOKEH_FIGS['x-px']       , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['y-py']       , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['zeta-pzeta'] , x_lim=(-1,1),y_lim=(-1,1), aspect=1, margin=0)
bktools.set_aspect(BOKEH_FIGS['x-y']        , x_lim=(-6,6),y_lim=(-6,6), aspect=1, margin=0)



grid = bklay.gridplot([[BOKEH_FIGS['x-px'] ,BOKEH_FIGS['y-py'] ,BOKEH_FIGS['zeta-pzeta']],[BOKEH_FIGS['x-y'],chunk_slider]],toolbar_location='right')
bk.show(grid)

In [ ]:
data.checkpoint

In [ ]:
# chunk_df = data.data[['Chunk ID','start_at_turn','stop_at_turn']].groupby('Chunk ID').mean()
# chunk_df.insert(0,'x',(chunk_df['stop_at_turn']+chunk_df['start_at_turn'])/2)

xy = ('x','px')
x,y = xy
to_source = bktools.source_from_groupby(data.checkpoint,by='Chunk ID',columns = [x,y])
to_source.insert(0,'particle',data.checkpoint.groupby('Chunk ID').get_group(0).particle)

In [ ]:
to_source

In [ ]:
data

In [ ]:
    chunk_df = data.data[['Chunk ID','start_at_turn','stop_at_turn']].groupby('Chunk ID').mean()

    chunk_df.insert(0,'x',(chunk_df['stop_at_turn']+chunk_df['start_at_turn'])/2)


    _df_list = []
    for col in ['x']:
        _df = pd.DataFrame({f'{_key}:{col}':_group[col].values for _key,_group in chunk_df.groupby('Chunk ID')})
        _df_list.append(_df)

    to_source = pd.concat(_df_list,axis=1)
    for col in ['x'][::-1]:
        to_source.insert(0,f'active:{col}',to_source[f'0:{col}'])
    
    to_source.insert(0,'width',(chunk_df['stop_at_turn']-chunk_df['start_at_turn']).max())
    to_source.insert(1,'height',1)
    to_source.insert(2,'y',0.5)

In [ ]:
to_source

In [ ]:
# Extraction action
J_df = data.checkpoint[['BUNCH','Chunk ID','turn','particle']]
J_df.insert(4,'Jx/emitt',1/2 * (data.checkpoint_sig.x_sig**2 + data.checkpoint_sig.px_sig**2))
J_df.insert(5,'Jy/emitt',1/2 * (data.checkpoint_sig.y_sig**2 + data.checkpoint_sig.py_sig**2))
J_df.dropna(inplace=True)

# Making Hextile grid
J_min  = 0
J_max  = 100
n_bins = 300

_size        = (J_max-J_min)/n_bins
_orientation = 'pointytop'


# Creating hextile template
x_corners = [J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max()]
y_corners = [J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max()]
XX,YY    = np.meshgrid(np.arange(J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max(),_size),
                       np.arange(J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max(),_size))
hextiles_template = bkhex.hexbin(np.array(list(XX.flatten())+x_corners), np.array(list(YY.flatten())+y_corners), size=_size,orientation=_orientation)
hextiles_template['counts']  = 0
hextiles_template = hextiles_template.rename(columns={'counts':'counts:active'}).set_index(['q','r'])

# Looping over chunks
for name,group in J_df.groupby('Chunk ID'):
    # Forcing corner values to have same grid.
    _hex = bkhex.hexbin(np.array(list(group['Jx/emitt']) + [J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max()]),
                        np.array(list(group['Jy/emitt']) + [J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max()]),
                        size=_size,orientation=_orientation)

    # Removing corner values
    _hex = _hex[1:-1]

    # Adding chunk ID
    hextiles_template.insert(name+1,f'counts:{name}',_hex.set_index(['q','r'])['counts'])


# setting empty bins to 0
hextiles_template = hextiles_template.fillna(0).reset_index()
hextiles_template['counts:active'] = hextiles_template['counts:0']


In [ ]:
hextiles_template

In [ ]:
# Creating hextile template
x_corners = [J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max()]
y_corners = [J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max()]
XX,YY    = np.meshgrid(np.arange(J_df['Jx/emitt'].min(),J_df['Jx/emitt'].max(),_size),
                       np.arange(J_df['Jy/emitt'].min(),J_df['Jy/emitt'].max(),_size))
hextiles_template = bkhex.hexbin(np.array(list(XX.flatten())+x_corners), np.array(list(YY.flatten())+y_corners), size=_size,orientation=_orientation)
hextiles_template['counts']  = 0

In [ ]:
hextiles_template.rename(columns={'counts':'counts:active'}).set_index(['q','r'])

In [ ]:
bktools.source_from_groupby(hextiles,by='Chunk ID',columns = ['counts'])

In [ ]:
J_df['Jx/emitt'].argmin(),J_df['Jx/emitt'].max()

In [ ]:
_hex_to_delete

In [ ]:
_hex.iloc[[0,len(_hex)-1]] 

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
collection = plt.hexbin(group['Jx/emitt'],group['Jy/emitt'], bins=100)

In [ ]:

import rich
rich.inspect(collection)

In [ ]:
J_df = data.checkpoint[['BUNCH','Chunk ID','turn','particle']]
J_df.insert(4,'Jx/emitt',1/2 * (data.checkpoint_sig.x_sig**2 + data.checkpoint_sig.px_sig**2))
J_df.insert(5,'Jy/emitt',1/2 * (data.checkpoint_sig.y_sig**2 + data.checkpoint_sig.py_sig**2))
J_df.dropna(inplace=True)


# coll_sig = np.max([data.sig_x_coll,data.sig_y_coll])
coll_max = 10
_x_dummy = np.linspace(0,10**2/2 ,300)


_size        = np.min(np.abs(np.diff(_x_dummy)))
_orientation = 'pointytop'

hextiles = bkhex.hexbin(J_df['Jx/emitt'],J_df['Jy/emitt'], _size)

In [ ]:
J_df

In [ ]:
hextiles

In [ ]:
chunk_df = data.data[['Chunk ID','start_at_turn','stop_at_turn']]
chunk_df.insert(1,'width',chunk_df['stop_at_turn']-chunk_df['start_at_turn'])
chunk_df.insert(2,'height',1)
chunk_df.insert(3,'x',(chunk_df['stop_at_turn']+chunk_df['start_at_turn'])/2)
chunk_df.insert(4,'y',0)
chunk_df.groupby('Chunk ID').apply(lambda row: list(row['width']),axis=1)

In [ ]:
chunk_df.groupby('Chunk ID')['width'].apply(lambda group : list(group)).to_frame()

In [ ]:
chunk_df

In [ ]:
460000*7,20000*23

In [ ]:
chunk_df

In [ ]:

chunk_df = data.data[['Chunk ID','start_at_turn','stop_at_turn']].groupby('Chunk ID').mean()
chunk_df.insert(1,'width',chunk_df['stop_at_turn']-chunk_df['start_at_turn'])
chunk_df.insert(2,'height',1)
chunk_df.insert(3,'x',(chunk_df['stop_at_turn']+chunk_df['start_at_turn'])/2)
chunk_df.insert(4,'y',0)
# chunk_df

_df_list = []
for col in ['x','width']:
    _df = pd.DataFrame({f'{_key}:{col}':_group[col].values for _key,_group in chunk_df.groupby('Chunk ID')})
    _df_list.append(_df)

to_source = pd.concat(_df_list,axis=1)
for col in ['x','width'][::-1]:
    to_source.insert(0,f'active:{col}',to_source[f'0:{col}'])

    
source = bkto_source

In [ ]:
test[(2,1,1)]

In [ ]:

{key:group for key,group in groups}[0]

In [ ]:
J_df.dropna()

In [ ]:
sum(data.checkpoint.state==1)

In [ ]:
data.checkpoint.dropna()

In [ ]:

    
test = coordinate_table(data.checkpoint,data.W_matrix,data.particle_on_co)

     